In [1]:
import glob
import pickle
import numpy
from music21 import instrument, converter, note, chord
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation, BatchNormalization 
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
def GetNotes(midiPath):
    notes = []
    for file in glob.glob(midiPath):
        midiData = converter.parse(file)
        print("Parsing ", file, end="... ")
        rawNotes = None
        try:
            rawNotes = instrument.partitionByInstrument(midiData).parts[0].recurse() 
        except:
            rawNotes = midi.flat.notes
        for element in rawNotes:
            if type(element) == chord.Chord:
                notes.append('.'.join(str(n) for n in element.normalOrder))
            elif type(element) == note.Note:
                notes.append(str(element.pitch))
        print("Done.")    
    return notes

In [3]:
def PrepTrainDatas(notes, notesTypesCount, seqLen = 100):
    
    notesNum = dict((note, number) for number, note in enumerate(sorted(set(item for item in notes))))

    nInput = []
    nOutput = []

    for i in range(len(notes) - seqLen):
        nInput.append([notesNum[n] for n in notes[i:i + seqLen]])
        nOutput.append(notesNum[notes[i + seqLen]])
    nInput = numpy.reshape(nInput, (len(nInput), seqLen, 1))
    nInput = nInput / notesTypesCount
    nOutput = np_utils.to_categorical(nOutput)

    return nInput, nOutput


In [4]:
def Network(nInput, notesTypesCount):
    model = Sequential()
    model.add(LSTM(300, input_shape=(nInput.shape[1], nInput.shape[2]),
        recurrent_dropout=0.3, return_sequences=True))
    model.add(LSTM(300, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(300))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(100))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(notesTypesCount))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [5]:
#from IPython.display import SVG
#from keras.utils import model_to_dot

#model = Network()
#SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [6]:
def Train(midiPath, epos, bsize):
    notes = GetNotes(midiPath)
    notesTypesCount = len(set(notes))

    (nInput, nOutput) = PrepTrainDatas(notes, notesTypesCount)

    model = Network(nInput, notesTypesCount)
    
    filepath = "w.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0,
        save_best_only=True, mode='min')
    callbacksList = [checkpoint]

    model.fit(nInput, nOutput, epochs=epos, batch_size=bsize, callbacks=callbacksList)

In [7]:
Train("sus/*.mid", 150, 150)

Parsing  sus/ff4pclov.mid...Done.
Parsing  sus/Oppressed.mid...Done.
Parsing  sus/ff11_awakening_piano.mid...Done.
Parsing  sus/great_war.mid...Done.
Parsing  sus/OTD5YA.mid...Done.
Parsing  sus/Ff4-BattleLust.mid...Done.
Parsing  sus/Final_Fantasy_Matouyas_Cave_Piano.mid...Done.
Parsing  sus/ff4-town.mid...Done.
Parsing  sus/balamb.mid...Done.
Parsing  sus/Rydia_pc.mid...Done.
Parsing  sus/Eternal_Harvest.mid...Done.
Parsing  sus/sobf.mid...Done.
Parsing  sus/BlueStone_LastDungeon.mid...Done.
Parsing  sus/lurk_in_dark.mid...Done.
Parsing  sus/In_Zanarkand.mid...Done.
Parsing  sus/bcm.mid...Done.
Parsing  sus/FF4.mid...Done.
Parsing  sus/ViviinAlexandria.mid...Done.
Parsing  sus/Ff7-Cinco.mid...Done.
Parsing  sus/ff1battp.mid...Done.
Parsing  sus/Fyw_piano.mid...Done.
Parsing  sus/ff4-airship.mid...Done.
Parsing  sus/costadsol.mid...Done.
Parsing  sus/dontbeafraid.mid...Done.
Parsing  sus/ultros.mid...Done.
Parsing  sus/ff7themep.mid...Done.
Parsing  sus/pkelite4.mid...Done.
Parsing  s

KeyboardInterrupt: 